In [20]:
import numpy as np
import matplotlib.pyplot as plt
import csv

In this case, we will try two lane roads.

In [16]:
class Vehicle:
    def __init__(self, vehicle_id, x_position, y_position, speed, max_speed, braking_prob, length, width):
        self.vehicle_id = vehicle_id #for vehicle identification
        self.x_position = x_position #position of the rear bumper
        self.y_position = y_position #position of the right side of the vehicle
        self.speed = speed #number of cells the vehicle moves per timestep
        self.max_speed = max_speed #maximum speed allowed for a vehicle
        self.braking_prob = braking_prob #probability of braking
        self.length = length #x dimension of vehicle
        self.width = width #y dimension of vehicle

    def accelerate(self):
        if self.speed < self.max_speed:
            self.speed += 1 

    def decelerate(self, gap_distance):
        if self.speed > gap_distance:
            self.speed = gap_distance

    def randomize(self, braking_prob):
        if np.random.rand() < braking_prob:
            if self.speed > 0:
                self.speed -= 1

In [17]:
class Road_Configuration:
    def __init__(self, road_length, road_width, production_prob, max_speed):
        self.road_length = road_length
        self.road_width = road_width 
        self.production_prob = production_prob #probability of a vehicle appearing on the start of the road
        self.max_speed = max_speed
        self.road_occupancy = np.zeros((self.road_length, self.road_width), dtype=int) #initialize a state of road occupancy
        self.vehicles = [] #initialize an empty list of vehicles
        self.vehicle_counter = 0  # To assign unique IDs to vehicles

    def update_occupancy(self):
        self.road_occupancy.fill(0)  # Reset occupancy grid
        for vehicle in self.vehicles:
            if vehicle.x_position < self.road_length and vehicle.y_position < self.road_width:  # Only update occupancy if within road length and width
                x_start = vehicle.x_position
                x_end = min(vehicle.x_position + vehicle.length, self.road_length)
                y_start = vehicle.y_position
                y_end = min(vehicle.y_position + vehicle.width, self.road_width)
                self.road_occupancy[x_start:x_end, y_start:y_end] = 1  # Assuming single-lane for simplicity


    def produce_vehicles(self):
        if np.random.rand() < self.production_prob: #Determines the time to produce vehicles
            vehicle_length = np.random.choice([1,2,5]) #vehicle length is selected (1 for motorcycle, 2 for four-wheeled car, 5 for trucks)
            if vehicle_length == 1:
                vehicle_width = 1 #Motorcycles have a dimension of 1x1
            else:
                vehicle_width = 2 #other vehicles have a dimension of 2x2, 2x5
            '''Pick a lane for a vehicle produced'''
            if vehicle_width == 1: #for motorcycles, the starting y position could either be of the two lanes
                initial_lane = np.random.choice([0,1,2,3])
            else:
                initial_lane = np.random.choice([0,1,2]) #starting y position for the other vehicles is always at the rightmost lane(all of them have a width of 2)
                    
            if np.sum(self.road_occupancy[0:vehicle.length-1, 0:vehicle.width-1]) == 0:  # Check the availability of the first cell
                new_vehicle = Vehicle(self.vehicle_counter, 0, initial_lane, np.random.randint(1, self.max_speed + 1), self.max_speed, 0.01, vehicle_length, vehicle_width)
                self.vehicle_counter += 1
                self.vehicles.append(new_vehicle)
                self.road_occupancy[0:new_vehicle.length, initial_lane:new_vehicle.width] = 1  # Mark the position as occupied

    def gap_distance(self, vehicle): #calculate headway distance
        current_x_position = vehicle.x_position #
        for distance in range(1, self.road_length): #checks the cells 1 step from the current position to the end of the road
            next_x_position = current_x_position + vehicle.length + distance
            if next_x_position >= self.road_length:
                break
            if self.road_occupancy[next_x_position:vehicle.length -1, 0:vehicle.width -1] == 1: #June 26, 2024, 5:02 PM (continue here)
                return distance - 1
        return self.road_length - 1

    def update(self): #updates the road configuration based on nagel-schreckenberg rules
        self.update_occupancy()
        for vehicle in self.vehicles:
            distance_to_next = self.gap_distance(vehicle)
            vehicle.accelerate()
            vehicle.decelerate(distance_to_next)
            vehicle.randomize(vehicle.braking_prob)
            vehicle.x_position += vehicle.speed

        self.update_occupancy()

Here, I am trying to simulate an intersection. One road before the intersection, and one road after it. 

In [19]:
class Intersection_Organizer: # In this simulation, I am excluding data extraction from the simulation
    def __init__(self, roads):
        self.roads = roads
        self.occupancy_history = []

    def transfer_vehicle(self, vehicle, destination_road):
        vehicle.x_position = 0 #Reset position to the beginning of the road
        destination_road.vehicles.append(vehicle)
        destination_road.update_occupancy()

    def update_intersection(self):
        #Define the roads before the intersection, the node road, and the roads after the intersection
        road_before_intersection = self.roads[0]
        node_road = self.roads[1]
        road_after_intersection = self.roads[2]

        #The road before the intersection produces vehicles on its beginning
        road_before_intersection.produce_vehicles()

        #Update all roads
        for road in self.roads:
            road.update()

        #Transfer vehicles to the intersection
        vehicles_to_transfer = [vehicle for vehicle in road_before_intersection.vehicles if vehicle.x_position >=road_before_intersection.road_length]
        for vehicle in vehicles_to_transfer:
            self.transfer_vehicle(vehicle, node_road)
        road_before_intersection.vehicles = [vehicle for vehicle in road_before_intersection.vehicles if vehicle.x_position < road_before_intersection.road_length]

        #Transfer vehicles from the intersection to the road after the intersection
        for vehicle in node_road.vehicles[:]:
            if vehicle.x_position >= node_road.road_length:
                if vehicle.width == 1: #for motorcycles, the starting y position could either be of the two lanes
                    initial_lane = np.random.choice([0,1,2,3])
                else:
                    initial_lane = np.random.choice([0,1,2]) #starting y position for the other vehicles is always at the rightmost lane(all of them have a width of 2)
                if np.sum(road_after_intersection.road_occupancy[0:vehicle.length, initial_lane:vehicle.width]) == 0: #Check the availability of cells corresponding to vehicle dimensions
                    self.transfer_vehicle(vehicle, road_after_intersection)
                    node_road.vehicles = [vehicle for vehicle in node_road.vehicles if vehicle.vehicle_id != vehicle.vehicle_id]


        self.occupancy_history.append([road.road_occupancy.copy() for road in self.roads])

    def save_occupancy_to_csv(self, filename):
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            for timestep, occupancy in enumerate(self.occupancy_history):
                for road_id, road_occupancy in enumerate(occupancy):
                    flattened_occupancy = road_occupancy.flatten()
                    writer.writerow([timestep, road_id] + flattened_occupancy.tolist())